## TrendAI

#### The TrendAI feature uses Generative AI for image generation. The stable diffusion model is used to generate designs according to the prompts given. The inputs such as garment type, color, fabrice and trend are taken from the user. The output image generated based on these prompts. 

##### Installing the libraries

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy peft

##### Setting up an image generation model and loading special weights to improve its performance.
(Use GPU to get faster results)

In [ ]:
import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline, TCDScheduler
from huggingface_hub import hf_hub_download

# Choose either 8 or 12 steps:
num_inference_steps = 12

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
repo_name = "ByteDance/Hyper-SD"
plural = "s" if num_inference_steps > 1 else ""
ckpt_name = f"Hyper-SDXL-{num_inference_steps}step{plural}-CFG-lora.safetensors"
device = "cuda"

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.load_lora_weights(hf_hub_download(repo_name, ckpt_name))
pipe.fuse_lora()
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

##### Entering the prompt. The output image is displayed.

In [ ]:
prompt = "A long red satin dress"
seed = random.randint(0, sys.maxsize)

# Pick a value between 5.0 and 8.0:
guidance_scale = 5.0

# Decrease eta (min: 0, max: 1.0) to get more details with multi-step inference:
eta = 0.5

images = pipe(
    prompt = prompt,
    num_inference_steps = num_inference_steps,
    guidance_scale = guidance_scale,
    eta = eta,
    generator = torch.Generator(device).manual_seed(seed),
    ).images

print(f"Prompt:\t{prompt}\nSeed:\t{seed}")
media.show_images(images)
images[0].save("output.jpg")

#### References:
https://github.com/woctezuma/stable-diffusion-colab